In [89]:
import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

PROVIAMO IL NOSTRO DATASET

In [4]:
multiclass_test = pd.read_csv(r'C:\Users\Fra\Documents\SDS\SML\DATASET PROGETTO\multiclass_test.csv',header= 0)
multiclass_training = pd.read_csv(r'C:\Users\Fra\Documents\SDS\SML\DATASET PROGETTO\multiclass_training.csv', header= 0)

In [ ]:
#y_test

In [134]:
y_test = pd.get_dummies(multiclass_test, columns = ['label']).iloc[:,17395:17405]
x_train = np.array(multiclass_training.drop(['label','Unnamed: 0','DepMap_ID'], axis = 1))
y_train = pd.get_dummies(multiclass_training, columns = ['label']).iloc[:,17395:17405]
x_test = np.array(multiclass_test.drop(['label','DepMap_ID','Unnamed: 0'], axis = 1))


In [56]:

x_train_reshaped = x_train.reshape(817,17393)
x_test_reshaped = x_test.reshape(205,17393)

In [74]:
#np.shape(x_test_reshaped)

(205, 17393)

PRIMO MODELLO

In [94]:
model = Sequential()

model.add(Dense(200, input_shape=(17393,)))
model.add(Activation('relu'))
#model.add(Dropout(0.25))


model.add(Dense(10))
model.add(Activation('softmax'))
#categorical_crossentropy loss per classificazione (penalizza molto le previsioni che sono sbagliate ma con alta probabilità)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) #ottimizzatore che usa stochastic gradient , errore assoluto medio

early_stopping_monitor = EarlyStopping(patience= 5, monitor = 'val_accuracy')

model_chekpoint_callback = ModelCheckpoint(filepath='best_model.hdf5',
                                           monitor = 'val_accuracy',
                                           mode = 'auto',
                                           verbose=1)

hystory = model.fit(x_train_reshaped, y_train,
         validation_data = (x_test_reshaped,y_test),
                    epochs =30,
                   callbacks = (early_stopping_monitor,model_chekpoint_callback))

#in multiclass possiamo aggiungere la softmax come activation function in modo da aumentare la probabilità che solo
#un nodo di uscita venga scelto ogni volta
#se per caso due possibili risultati sono anche parzialmente sovrapponibili si usa come activation function sigmoid

Epoch 1/30
25/26 [===========================>..] - ETA: 0s - loss: 3.0643 - accuracy: 0.2175
Epoch 00001: saving model to best_model.hdf5
26/26 [==============================] - 1s 22ms/step - loss: 3.0395 - accuracy: 0.2203 - val_loss: 1.9617 - val_accuracy: 0.3024
Epoch 2/30
22/26 [========================>.....] - ETA: 0s - loss: 1.8724 - accuracy: 0.3892
Epoch 00002: saving model to best_model.hdf5
26/26 [==============================] - 0s 16ms/step - loss: 1.8370 - accuracy: 0.4039 - val_loss: 1.7956 - val_accuracy: 0.3317
Epoch 3/30
25/26 [===========================>..] - ETA: 0s - loss: 1.5421 - accuracy: 0.4787
Epoch 00003: saving model to best_model.hdf5
26/26 [==============================] - 0s 17ms/step - loss: 1.5368 - accuracy: 0.4810 - val_loss: 1.7746 - val_accuracy: 0.3268
Epoch 4/30
23/26 [=========================>....] - ETA: 0s - loss: 1.4143 - accuracy: 0.5421
Epoch 00004: saving model to best_model.hdf5
26/26 [==============================] - 0s 16ms/step 

SECONDO MODELLO: 

aggiungiamo un layer e raddoppiamo i nodi per layer

aumentiamo il numero di epoche quando aggiungiamo un layer

In [121]:
model = Sequential()

#1°layer
model.add(Dense(1000, input_shape=(17393,)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#2°layer
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.2))

#3°layer
model.add(Dense(10))
model.add(Activation('softmax'))

#categorical_crossentropy loss per classificazione (penalizza molto le previsioni che sono sbagliate ma con alta probabilità)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) #ottimizzatore che usa stochastic gradient , errore assoluto medio

early_stopping_monitor = EarlyStopping(patience= 15, monitor = 'val_accuracy')

model_chekpoint_callback = ModelCheckpoint(filepath='best_model.hdf5',
                                           monitor = 'val_accuracy',
                                           mode = 'auto',
                                           save_best_only=True,
                                           verbose=1)

hystory = model.fit(x_train_reshaped, y_train,
         validation_data = (x_test_reshaped,y_test),
                    epochs =30,
                   callbacks = (early_stopping_monitor,model_chekpoint_callback))


Epoch 1/30
26/26 [==============================] - ETA: 0s - loss: 3.4897 - accuracy: 0.1750
Epoch 00001: val_accuracy improved from -inf to 0.31220, saving model to best_model.hdf5
26/26 [==============================] - 3s 88ms/step - loss: 3.4897 - accuracy: 0.1750 - val_loss: 2.2167 - val_accuracy: 0.3122
Epoch 2/30
26/26 [==============================] - ETA: 0s - loss: 2.0731 - accuracy: 0.2889
Epoch 00002: val_accuracy improved from 0.31220 to 0.32683, saving model to best_model.hdf5
26/26 [==============================] - 2s 85ms/step - loss: 2.0731 - accuracy: 0.2889 - val_loss: 1.9112 - val_accuracy: 0.3268
Epoch 3/30
26/26 [==============================] - ETA: 0s - loss: 1.7977 - accuracy: 0.3733
Epoch 00003: val_accuracy improved from 0.32683 to 0.48293, saving model to best_model.hdf5
26/26 [==============================] - 2s 84ms/step - loss: 1.7977 - accuracy: 0.3733 - val_loss: 1.6390 - val_accuracy: 0.4829
Epoch 4/30
26/26 [==============================] - ETA

In [122]:
#To predict only the best model

from keras.models import load_model

best_model = load_model('best_model.hdf5')
y_pred = np.argmax(best_model.predict(x_test_reshaped),axis = 1)
y_pred = pd.get_dummies(y_pred)

In [138]:
#Create the vectors that contains the Cancers chosen (Cancer_Type2) and the one observed (Cancer_Type)

Cancer_Type = ["Gastrointestinal","Genitals","Muscle-Bone","Neuro","Breast","Head_Neck","Blood","Urinario","Lung"]
Cancer_Type2 = ["Others","Gastrointestinal","Genitals","Muscle-Bone","Neuro","Breast","Head_Neck","Blood","Urinario","Lung"]

y_test_new = y_test.drop(['label_0'],axis = 1) #because in the prediction Others doesn't show up


In [139]:
#Change the name of the coloumns of the class matrix created through get_dummies using more meaningful names

y_test_new = y_test_new.set_axis(Cancer_Type, axis = 1)
y_pred_new = y_pred.set_axis(Cancer_Type, axis= 1)
y_train_new =  y_train.set_axis(Cancer_Type2, axis= 1)
y_test_new2 = y_test.set_axis(Cancer_Type2, axis = 1)

In [141]:
#To observe the distribution of the observation w.r.t. CancerType

#y_train_new.sum() 
#y_test_new2.sum()

Gastrointestinal    46
Genitals            22
Muscle-Bone         21
Neuro               31
Breast               7
Head_Neck           14
Blood               19
Urinario            12
Lung                30
dtype: int64

In [107]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test_new, y_pred_new)

array([[[140,  19],
        [  8,  38]],

       [[172,  11],
        [  9,  13]],

       [[178,   6],
        [  7,  14]],

       [[153,  21],
        [  4,  27]],

       [[198,   0],
        [  2,   5]],

       [[188,   3],
        [  6,   8]],

       [[185,   1],
        [  0,  19]],

       [[191,   2],
        [  8,   4]],

       [[170,   5],
        [ 21,   9]]], dtype=int64)

In [119]:
y_pred = np.argmax(best_model.predict(x_test_reshaped),axis = 1)
y_test_confusion = multiclass_test.iloc[:,17395]
np.unique(y_test_confusion)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [117]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_confusion,y_pred)

array([[ 0,  0,  0,  2,  1,  0,  0,  0,  0,  0],
       [ 0, 38,  3,  1,  3,  0,  1,  0,  0,  0],
       [ 0,  3, 13,  0,  3,  0,  0,  0,  1,  2],
       [ 0,  1,  1, 14,  4,  0,  1,  0,  0,  0],
       [ 0,  0,  2,  1, 27,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  2],
       [ 0,  2,  1,  0,  3,  0,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 19,  0,  0],
       [ 0,  3,  2,  0,  2,  0,  0,  1,  4,  0],
       [ 0, 10,  2,  2,  5,  0,  1,  0,  1,  9]], dtype=int64)

In [ ]:
Gastrointestinal == 1

Genitals == 2

Muscle-Bone == 3

Neuro == 4

Breast == 5

Head_Neck == 6

Blood == 7

Urinario == 8

Lung == 9

Others == 0

for multiclass

def target_encode_multiclass(X,y): #X,y are pandas df and series   
    y=y.astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)    
    class_names=y_onehot.columns  #names of onehot encoded columns    X_obj=X.select_dtypes('object') #separate categorical columns
    X=X.select_dtypes(exclude='object')     
    for class_ in class_names:
        enc=ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(X_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        X=pd.concat([X,temp],axis=1)    #add to original dataset
      
    return X

In [ ]:
##for i in range(len(Cancer_Type)):
  #  y_test = y_test.rename(columns={"label_" + str(i+1):Cancer_Type[i]}, errors= 'raise')